In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.metrics import f1_score, ConfusionMatrixDisplay, roc_auc_score, roc_curve
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.ensemble import RandomForestClassifier, \
GradientBoostingClassifier, VotingClassifier
from sklearn.inspection import permutation_importance

In [2]:
df = pd.read_csv('AviationData.csv', encoding='latin1')
print(df.head())

         Event.Id Investigation.Type Accident.Number  Event.Date  \
0  20001218X45444           Accident      SEA87LA080  1948-10-24   
1  20001218X45447           Accident      LAX94LA336  1962-07-19   
2  20061025X01555           Accident      NYC07LA005  1974-08-30   
3  20001218X45448           Accident      LAX96LA321  1977-06-19   
4  20041105X01764           Accident      CHI79FA064  1979-08-02   

          Location        Country   Latitude  Longitude Airport.Code  \
0  MOOSE CREEK, ID  United States        NaN        NaN          NaN   
1   BRIDGEPORT, CA  United States        NaN        NaN          NaN   
2    Saltville, VA  United States  36.922223 -81.878056          NaN   
3       EUREKA, CA  United States        NaN        NaN          NaN   
4       Canton, OH  United States        NaN        NaN          NaN   

  Airport.Name  ... Purpose.of.flight Air.carrier Total.Fatal.Injuries  \
0          NaN  ...          Personal         NaN                  2.0   
1         

C:\Users\basne\AppData\Local\Temp\ipykernel_10264\794488422.py:1: DtypeWarning: Columns (6,7,28) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('AviationData.csv', encoding='latin1')


In [3]:
df.shape

(87282, 31)

In [4]:
df.isnull().sum().apply(lambda x: x/87282*100)

Event.Id                   0.000000
Investigation.Type         0.000000
Accident.Number            0.000000
Event.Date                 0.000000
Location                   0.059577
Country                    0.258931
Latitude                  62.107880
Longitude                 62.118192
Airport.Code              43.491212
Airport.Name              40.474554
Injury.Severity            1.011663
Aircraft.damage            3.448592
Aircraft.Category         64.731560
Registration.Number        1.508902
Make                       0.072180
Model                      0.105405
Amateur.Built              0.116863
Number.of.Engines          6.586696
Engine.Type                6.749387
FAR.Description           65.103916
Schedule                  85.873376
Purpose.of.flight          6.632524
Air.carrier               81.701840
Total.Fatal.Injuries      13.062258
Total.Serious.Injuries    14.332852
Total.Minor.Injuries      13.671777
Total.Uninjured            6.773447
Weather.Condition          4

In [5]:
print(set(df['Make'].unique()))

{'Nicolosi', 'SilverLight Aviation', 'Mccaa', 'U-FLY-IT', 'Head Balloons, Inc.', 'Robert C. Campbell', 'Humer', 'Donegan Benton', 'Kenneson', 'Hammack', 'Eris E. Ervin', 'Birx', 'Beggs', 'Nord Aviation', 'WINGS AN THINGS INC', 'QUICKSILVER MANUFACTURING INC', 'Nicks', 'TEAM ROCKET F-1', 'GREMMINGER GREG', 'Widner', 'Bowers Flybaby', 'Oneil', 'Hesler', 'AEROSTAR S A', 'Baker, Timithy', 'Arctic Aircraft Co. Inc.', 'Md Helicopters, Inc.', 'Cessna Ector', 'Paulistinha  56-C1', 'Mooney', 'Frennance', 'ANDREWS', 'Landes/karich/schmidt', 'Funk, D.d. Aviation Co.', 'Dassault Aviation', 'ROSS H/HERRIOTT M', 'Patch Dan F', 'Simoneau', 'DOKTOR FIBERGLAS', 'Questair, Inc.', 'Trenti', 'Cianchette', 'Cameron Balloons US', 'Corlett-beahm', 'Auton - Dunham', 'Mason, Jack A.', 'Columbia', 'QUESTAIR INC', 'Bombardier Aerospace, Inc.', 'AIR TRACTOR INC', 'MCLEOD ALAN', 'ARADO-FLUGZEUGWERKE GMBH', 'Kainz', 'Silveira Jonathan A', 'VANDERHOOF STEVE', 'Huntley', 'Carris', 'Rossi Pitts', 'Lounsbery', 'Eaa Cha

In [6]:
cols = ['Investigation.Type', 'Accident.Number', 'Event.Date', 'Country',
        'Injury.Severity','Aircraft.damage', 'Make', 'Purpose.of.flight',
        'Total.Fatal.Injuries', 'Total.Serious.Injuries', 'Total.Minor.Injuries',
        'Total.Uninjured', 'Weather.Condition']
df1 = df[cols]
df1.to_csv('aviation1.csv', index=False, encoding='utf-8')

In [7]:
df1.shape

(87282, 13)

In [8]:
cols = ['Total.Fatal.Injuries', 'Total.Serious.Injuries',
        'Total.Minor.Injuries', 'Total.Uninjured']
df1[cols] = df1[cols].fillna(0)

C:\Users\basne\AppData\Local\Temp\ipykernel_10264\3508871344.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1[cols] = df1[cols].fillna(0)


In [9]:
df1.isnull().sum()

Investigation.Type           0
Accident.Number              0
Event.Date                   0
Country                    226
Injury.Severity            883
Aircraft.damage           3010
Make                        63
Purpose.of.flight         5789
Total.Fatal.Injuries         0
Total.Serious.Injuries       0
Total.Minor.Injuries         0
Total.Uninjured              0
Weather.Condition         3978
dtype: int64